We will use a dataset of general tweets (double-check where it's from) for training, and a dataset of hand-labeled customer feedback to airlines for validation



# TODO Update this

This has only been trained on a small amount of the data. Training on all of the data takes ~30 hours.

In [11]:
import tweepy

In [12]:
from twitter_nlp_toolkit.twitter_listener import twitter_listener

In [13]:
consumer_key = "CawRoWHuFDzS12iprfvANhatY"
consumer_secret = "IXMJ75QNAYQfyyoOwTeJeFLei1uVymg2YsqGdojthbAcv54aI3"
access_token = "826992578075648002-CrELY6bpnPoyyMqIBq2xm9myLiOChWH"
access_secret = "3etfQ9nTTg46DlrxU4HnXD2PkETuRrNN4Un8KJVOMdkrD"

In [14]:
stream1 = twitter_listener.twitter_stream_listener(consumer_key,consumer_secret,access_token,access_secret,["Trump","Impeachment"])

In [15]:
stream1.collect_from_stream(100,"test.json",1)


Initializing listener... 


AttributeError: 'twitter_stream_listener' object has no attribute 'auth'